In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib qt

# Imports

In [ ]:
import jax

# Get the percision of the jax library
print()
print('Jax has been configured to use the following devices: ', jax.devices())
jax.config.update("jax_enable_x64", True)

In [ ]:
import numpy as np
from ICARUS.propulsion.engine import Engine
from ICARUS.database import DB

#  Load Plane and Engine

In [ ]:
engine_dir = "../Data/Engine/Motor_1/"

engine = Engine()
engine.load_data_from_df(engine_dir)
# engine.plot_engine_map()

# Vehicle

In [ ]:
from ICARUS.vehicle.plane import Airplane
from ICARUS.mission.mission_vehicle import MissionVehicle

In [ ]:
plane: Airplane = DB.get_vehicle('final_design')
# plane.visualize(annotate=True)
# plane.main_wing.symmetries

mission_plane = MissionVehicle(
    plane,
    engine,
    solver= "AVL"
)

# from ICARUS.visualization.airplane.db_polars import plot_airplane_polars
# plot_airplane_polars([plane.name])

# Compute Trajectory

In [ ]:
from ICARUS.geometry.polynomial import h_polynomial_factory
from ICARUS.mission.trajectory.trajectory import MissionTrajectory
from ICARUS.mission.trajectory.integrate import RK45_scipy_integrator
from ICARUS.geometry.cubic_splines import CubicSpline_factory
import jax.numpy as jnp

In [ ]:
operating_floor = 12.5
t0 = 0
x0 =  jnp.array([0., 20.])

# Polynomial Trajectory
coeffs = jnp.array(
    [
        x0[1],
        0.2,
        -1/300
    ]
)
traj_fun_1 = h_polynomial_factory(coeffs)

traj_poly  = MissionTrajectory(
    "Polynomial Trajectory", 
    traj_fun_1, 
    vehicle=mission_plane,
    verbosity= 2,
    operating_floor= 12.4
)

# Spline Trajectory
y= [20.34201669, 18.79171704 ,20.01380251 ,20.70916841, 20.42523286 ,19.08487198]
# y = [20, 20 , 20, 20, 20, 20,]
x =  jnp.linspace(0, 2000, len(y)+2)
y = jnp.array([x0[1], *y, operating_floor])
spline_i, title = CubicSpline_factory(x,y)

traj_spl = MissionTrajectory(
    title, 
    spline_i, 
    vehicle=mission_plane,
    verbosity= 2,
    operating_floor= 12.4
)
gamma = jnp.arctan(traj_spl.dy_dx(x0[0]))


In [ ]:
import jax.numpy as jnp

from ICARUS.mission.trajectory.integrate import RK4systems
# velocity is in the same direction as the derivative of the polynomial
g1 = jnp.arctan(
    traj_spl.dy_dx(x0[0])
)
v0_mag = 20.
v0 = jnp.array([jnp.cos(g1), jnp.sin(g1)]) * v0_mag

dt = 0.01
tend = 200
t, xs, vs, states = RK4systems(t0, tend, dt, x0, v0, traj_spl )


In [ ]:
traj_spl.clear_history()
for statei, xi, vi, ti in zip(states, xs, vs, t):
    traj_spl.record_state(ti, xi, vi, *statei)
traj_spl.plot_history()

# Optimization

In [ ]:
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('Qt5Agg')


# fig = plt.figure()
# axs = fig.subplots(3, 2, squeeze=False).flatten().tolist()
# # plt.show(block=True)

def objective_goal_splines(y, dt = 0.05):
    x =  jnp.linspace(0, 2000, len(y)+2)
    y = jnp.hstack([x0[1], y, operating_floor])

    spline_i, title = CubicSpline_factory(x,y)

    trajectory = MissionTrajectory(
        title, 
        spline_i, 
        vehicle=mission_plane,
        verbosity= 2,
        operating_floor= 12.4
    )
    gamma = jnp.arctan(trajectory.dy_dx(x0[0]))
    v0 = jnp.array([np.cos(gamma), np.sin(gamma)]) * v0_mag


    t, xs, vs, _  = RK4systems(t0, tend, dt, x0, v0, trajectory)
    # trajectory.plot_history(axs,fig)
    # x = [x[0] for x in xs]
    # Remove nan values
    x = xs[:, 0]
    x = x[~jnp.isnan(x)]
    print(f"Max x: {jnp.max(x)}")
    return - jnp.max(x)

In [12]:
from scipy.optimize import minimize
 
res_splines = minimize(
    objective_goal_splines,
    x0 = np.array([20, 20 , 20, 20, 20, 20,]),
    method='COBYLA',
    options={'disp': True, 'maxiter' : 3000},
)



Simulation Completed Successfully at time 200.0       Max Distance: 3363.0617825402746
Max x: 3363.0617825402746
Simulation Completed Successfully at time 200.0       Max Distance: 3362.773331019203
Max x: 3362.773331019203
Airplane Crash at step: 1581           Max Distance: 2038.1997166916562 
Max x: 2038.1997166916562
Simulation Completed Successfully at time 200.0       Max Distance: 3363.0128266447355
Max x: 3363.0128266447355
Simulation Completed Successfully at time 200.0       Max Distance: 3363.0617800131768
Max x: 3363.0617800131768
Simulation Completed Successfully at time 200.0       Max Distance: 3362.964301523955
Max x: 3362.964301523955
Simulation Completed Successfully at time 200.0       Max Distance: 3363.0690898490316
Max x: 3363.0690898490316
Airplane Crash at step: 1582           Max Distance: 2039.5431328525656 
Max x: 2039.5431328525656
Simulation Completed Successfully at time 200.0       Max Distance: 3362.996952670661
Max x: 3362.996952670661
Simulation Comple

In [13]:
print(res_splines.x)

[21.71875195 21.25796244 19.84735035 20.00174574 21.48924275 17.06880046]


In [14]:
y  = res_splines.x
x = jnp.linspace(0, 2000, len(y)+2)
y = jnp.hstack([x0[1], y, operating_floor])

spline_best, title = CubicSpline_factory(x,y)
trajectory_best =  MissionTrajectory(
        title, 
        spline_best, 
        vehicle=mission_plane,
        verbosity= 2,
        operating_floor= 12.4
    )
gamma = jnp.arctan( trajectory_best.dy_dx(x0[0]) )
v0 = jnp.array([np.cos(gamma), np.sin(gamma)]) * v0_mag

t, xs, vs, states = RK4systems(t0, tend, dt, x0, v0, trajectory_best)

for statei, xi, vi, ti in zip(states, xs, vs, t):
    trajectory_best.record_state(ti, xi, vi, *statei)
# Plot Trajectory
trajectory_best.clear_history()
for statei, xi, vi, ti in zip(states, xs, vs, t):
    trajectory_best.record_state(ti, xi, vi, *statei)

trajectory_best.plot_history()

Airplane Crash at step: 7912	Time 79.12
	Max Distance: 2039.0059720361128 
Appending State
Appending State
Appending State
Appending State
Appending State
Appending State
Appending State
Appending State
Appending State
Appending State
Appending State
Appending State
Appending State
Appending State
Appending State
Appending State
Appending State
Appending State
Appending State
Appending State
Appending State
Appending State
Appending State
Appending State
Appending State
Appending State
Appending State
Appending State
Appending State
Appending State
Appending State
Appending State
Appending State
Appending State
Appending State
Appending State
Appending State
Appending State
Appending State
Appending State
Appending State
Appending State
Appending State
Appending State
Appending State
Appending State
Appending State
Appending State
Appending State
Appending State
Appending State
Appending State
Appending State
Appending State
Appending State
Appending State
Appending State
Appending Sta